In [ ]:
import pyodbc
import pandas as pd
import random
import sys
from datetime import datetime, timedelta
import threading 
import time
from time import sleep

In [ ]:
# creating connections with sql server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=STEELDNA;'
                      'Database=DeS;'
                      'UID=sa;'
                      'PWD=admin@123;'
                      'Trusted_Connection=no;')

cursor = conn.cursor()

In [7]:
# getting system starttime
now = datetime.now()

In [4]:
class RepeatedTimer(object):
  def __init__(self, interval, function, *args, **kwargs):
    self._timer = None
    self.interval = interval
    self.function = function
    self.args = args
    self.kwargs = kwargs
    self.is_running = False
    self.next_call = time.time()
    self.start()

  def _run(self):
    self.is_running = False
    self.start()
    self.function(*self.args, **self.kwargs)

  def start(self):
    if not self.is_running:
      self.next_call += self.interval
      self._timer = threading.Timer(self.next_call - time.time(), self._run)
      self._timer.start()
      self.is_running = True

  def stop(self):
    self._timer.cancel()
    self.is_running = False

In [147]:
def update_first_sample(status):
    
    # printing loop status
    print(status)
    
    # getting system starttime
    now = datetime.now()

    # creating connections with sql server
    conn = pyodbc.connect('Driver={SQL Server};'
                          'Server=STEELDNA;'
                          'Database=DeS;'
                          'UID=sa;'
                          'PWD=admin@123;'
                          'Trusted_Connection=no;')

    cursor = conn.cursor()

    
    # reading the table which has information regarding sample chemistry
    sample_chemistry_data= pd.read_sql("SELECT TOP(100) * FROM heat_analysis order by SAMPLE_TIME desc", conn)

    # keeping only data points from the last 3 hours
    # window selected based on analysis
    sample_chemistry_data = sample_chemistry_data[sample_chemistry_data['SAMPLE_TIME']>= now - timedelta(minutes = 180)]

    # first we remove all records of LF5, as we are not predicting for that LF
    sample_chemistry_data = sample_chemistry_data[sample_chemistry_data['AGGREGATE'] != 'LF5']

    # for this, first we have to find the heats which have the latest sample information as first sample
    sample_latest = sample_chemistry_data.groupby('HEAT_NUMBER').agg({'SAMPLE_CODE':'max'})
    sample_latest = sample_latest[sample_latest['SAMPLE_CODE'] == '25101']
    sample_latest.reset_index(inplace = True)

    # getting the latest heat numbers from master_output_table
    # we've already predicted the outputs for this heat, so we don't need to predict again
    previous_heats = pd.read_sql("SELECT TOP(100) HEAT_NUMBER FROM dbo.output_table ORDER BY MSG_TIME_STAMP desc",conn)
    previous_heats = list(previous_heats['HEAT_NUMBER'])
    previous_heats = [str(x) for x in previous_heats]
    
    # we will only run for any new heats that have been found
    latest_heats = sample_latest['HEAT_NUMBER'].unique()
    run_heats = [x for x in latest_heats if x not in previous_heats]
    
    # if this list is empty, we dont run the predictions code
    # if it is not empty, then we predict for the new heats
    if len(run_heats) != 0:
        
        
    
    ######################################################################################################################
    # this is a temporary testing logic
    # to just see if we can push to the output table regularly 
    # without having repeats in the heat number
    # this is the step where the actual regression equations would run
    # and the output would be stored in the master output table

    # test logic
    # to be replaced
    op_table = sample_latest.copy()
    op_table = op_table[~op_table['HEAT_NUMBER'].isin(heat_numbers)]
    op_table['NODE'] = [random.randint(1,16) for x in range(len(op_table))]
    op_table['INSERT_DATETIME'] = now
    op_table = op_table.merge(sample_chemistry_data, on = 'HEAT_NUMBER', how = 'left')
    # keeping only required columns for now
    op_table = op_table[['HEAT_NUMBER','SAMPLE_TIME','INSERT_DATETIME','NODE']]

    # writing to output_test_table
    for index, row in op_table.iterrows():
        cursor.execute("INSERT into dbo.master_output_table_test (HEAT_NUMBER,SAMPLE_TIME,INSERT_DATETIME,NODE) values(?,?,?,?)", row['HEAT_NUMBER'], row['SAMPLE_TIME'], row['INSERT_DATETIME'], row['NODE'])
        
    # closing cursor
    conn.commit()
    cursor.close()

In [154]:
print("starting...")
rt = RepeatedTimer(30, update_first_sample,"Started") # it auto-starts, no need of rt.start()
try:
    sleep(5)# your long-running job goes here...
finally:
    #rt.stop() # better in a try/finally block to make sure the program ends!
    print("failed")
    

starting...
Started
Started
failed
Started
Started
Started
Started


In [8]:
# creating connections with sql server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=STEELDNA;'
                      'Database=DeS;'
                      'UID=sa;'
                      'PWD=admin@123;'
                      'Trusted_Connection=no;')

cursor = conn.cursor()

In [9]:
# getting system starttime
now = datetime.now()

In [20]:
# reading the table which has information regarding sample chemistry
sample_chemistry_data= pd.read_sql("SELECT TOP(100) * FROM heat_analysis order by SAMPLE_TIME desc", conn)

# keeping only data points from the last 3 hours
# window selected based on analysis
sample_chemistry_data = sample_chemistry_data[sample_chemistry_data['SAMPLE_TIME']>= now - timedelta(minutes = 180)]

# first we remove all records of LF5, as we are not predicting for that LF
sample_chemistry_data = sample_chemistry_data[sample_chemistry_data['AGGREGATE'] != 'LF5']

# for this, first we have to find the heats which have the latest sample information as first sample
sample_latest = sample_chemistry_data.groupby('HEAT_NUMBER').agg({'SAMPLE_CODE':'max'})
sample_latest = sample_latest[sample_latest['SAMPLE_CODE'].isin(['25101','25201'])]
sample_latest.reset_index(inplace = True)

# getting the latest heat numbers from master_output_table
# we've already predicted the outputs for this heat, so we don't need to predict again
previous_heats = pd.read_sql("SELECT TOP(100) HEAT_NUMBER FROM dbo.output_table ORDER BY MSG_TIME_STAMP desc",conn)
previous_heats = list(previous_heats['HEAT_NUMBER'])
previous_heats = [str(x) for x in previous_heats]

# we will only run for any new heats that have been found
latest_heats = sample_latest['HEAT_NUMBER'].unique()
run_heats = [x for x in latest_heats if x not in previous_heats]


In [21]:
run_heats

[]

In [23]:
now = datetime.now().second

In [24]:
now

58